In [2]:
import numpy as np
from urllib.request import urlopen
import cv2
from bs4 import BeautifulSoup
import urllib.request
import sys                                          # System bindings
import pandas as pd
import operator

In [3]:
file=pd.read_csv('Hot Products 01 Jul.csv',low_memory=0) #input csv file

In [4]:
file['link']='https://shopee.co.id/product/'+file['ShopID'].map(str)+"/"+file['ItemID'].map(str)+"/"
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    with urlopen(url) as resp:
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
    # return the image
    return image

In [5]:
products = file.link.head()
urls=[]
for product in products:
    print(product)
    with urlopen(product) as response:
        html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    link = str(soup.find(property="twitter:image"))
    url = link[15:-28]
    urls.append(url)
    print ("downloading %s" % (url))

#creating list image
list_img=[]
for i,url in enumerate(urls):
    tmp=[]
    tmp.append(urllib.request.urlretrieve(url, "image-%s.jpg" % i))
    list_img.append(tmp[0][0])
    
#define class for color analysis
class ColorAnalyser():
    def __init__(self, imageLoc):
        self.src = cv2.imread(imageLoc, 1)          # Reads in image source
        self.colors_count = {}                      # Empty dictionary container to hold the colour frequencies
 
    def count_colors(self):
        (channel_b, channel_g, channel_r) = cv2.split(self.src) # Splits image Mat into 3 color channels in individual 2D arrays
 
        channel_b = channel_b.flatten()                         # Flattens the 2D single channel array so as to make it easier to iterate over it
        channel_g = channel_g.flatten()                         #                   ""
        channel_r = channel_r.flatten()                         #                   ""
 
        for i in range(len(channel_b)):
            RGB = "(" + str(channel_r[i]) + "," + str(channel_g[i]) + "," + str(channel_b[i]) + ")"
            if RGB in self.colors_count:
                self.colors_count[RGB] += 1
            else:
                self.colors_count[RGB] = 1
 
        print("Colours counted")
 
    def show_colors(self):
        colors={}
        #self.count_colors()
        for keys in sorted(self.colors_count, key=self.colors_count.__getitem__):       # Sorts dictionary by value
            colors.update({keys : self.colors_count[keys]})                  # save each new color detected in the dictionary 'key: value'
        return colors

    def main(self):
        if (self.src == None):                                  # Checks if an image was actually loaded and errors if it wasn't
            print("No image data. Check image location for typos")
        else:
            self.count_colors()                             # Counts the amount of instances of RGB values within the image
            self.show_colors()                              # Sorts and shows the colors ordered from least to most often occurance
            cv2.waitKey(0)                                  # Waits for keypress before closing

https://shopee.co.id/product/90309109/1904172659/
downloading https://cf.shopee.co.id/file/1daccd2d6ea99383c5e61f28875de10e
https://shopee.co.id/product/51115741/2167371343/
downloading https://cf.shopee.co.id/file/b2f29a0d2242a58f46749c682c9dee32
https://shopee.co.id/product/46567482/1153439532/
downloading https://cf.shopee.co.id/file/6a286c7751cdd8e63f8d0eecc6f05369
https://shopee.co.id/product/62582411/1022848316/
downloading https://cf.shopee.co.id/file/74f7f8dc50ebf4ad9b93f250dcb1d107
https://shopee.co.id/product/38181258/1464714627/
downloading https://cf.shopee.co.id/file/78723b51d19c62b7a21402418aae102d


In [6]:
if (len(sys.argv) != 2):                                    # Checks if image was given as cli argument
    print("error: syntax is 'python main.py /example/image/location.jpg'")
else:
    Analyser = ColorAnalyser(sys.argv[1])
    Analyser.main()

#determine the dominant color
analyser=[]
for img in list_img:
    an=ColorAnalyser(img)
    analyser.append(an)
for obj in analyser:
    obj.count_colors()

dominant_color=[]
for i in analyser:
    dominant_color.append(max(i.colors_count.items(),key=operator.itemgetter(1))[0])

find_dom=[]
for dom in dominant_color:
    if dom=='(255,255,255)':
        find_dom.append('White')
    else:
        find_dom.append('Non-White')

error: syntax is 'python main.py /example/image/location.jpg'
Colours counted
Colours counted
Colours counted
Colours counted
Colours counted


In [7]:
dominant_color

['(255,255,255)',
 '(255,255,255)',
 '(255,255,255)',
 '(255,255,255)',
 '(255,255,255)']

In [8]:
find_dom

['White', 'White', 'White', 'White', 'White']

In [14]:
len(analyser[0].show_colors())

14188